In [9]:
from langchain.schema import Document
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

from langchain_core.tools import tool
from langchain.tools.retriever import create_retriever_tool


from dotenv import load_dotenv
load_dotenv()

True

In [10]:
embedding_function = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

docs = [
    Document(
        page_content="Peak Performance Gym was founded in 2015 by former Olympic athlete Marcus Chen. With over 15 years of experience in professional athletics, Marcus established the gym to provide personalized fitness solutions for people of all levels. The gym spans 10,000 square feet and features state-of-the-art equipment.",
        metadata={"source": "about.txt"}
    ),
    Document(
        page_content="Peak Performance Gym is open Monday through Friday from 5:00 AM to 11:00 PM. On weekends, our hours are 7:00 AM to 9:00 PM. We remain closed on major national holidays. Members with Premium access can enter using their key cards 24/7, including holidays.",
        metadata={"source": "hours.txt"}
    ),
    Document(
        page_content="Our membership plans include: Basic (₹1,500/month) with access to gym floor and basic equipment; Standard (₹2,500/month) adds group classes and locker facilities; Premium (₹4,000/month) includes 24/7 access, personal training sessions, and spa facilities. We offer student and senior citizen discounts of 15% on all plans. Corporate partnerships are available for companies with 10+ employees joining.",
        metadata={"source": "membership.txt"}
    ),
    Document(
        page_content="Group fitness classes at Peak Performance Gym include Yoga (beginner, intermediate, advanced), HIIT, Zumba, Spin Cycling, CrossFit, and Pilates. Beginner classes are held every Monday and Wednesday at 6:00 PM. Intermediate and advanced classes are scheduled throughout the week. The full schedule is available on our mobile app or at the reception desk.",
        metadata={"source": "classes.txt"}
    ),
    Document(
        page_content="Personal trainers at Peak Performance Gym are all certified professionals with minimum 5 years of experience. Each new member receives a complimentary fitness assessment and one free session with a trainer. Our head trainer, Neha Kapoor, specializes in rehabilitation fitness and sports-specific training. Personal training sessions can be booked individually (₹800/session) or in packages of 10 (₹7,000) or 20 (₹13,000).",
        metadata={"source": "trainers.txt"}
    ),
    Document(
        page_content="Peak Performance Gym's facilities include a cardio zone with 30+ machines, strength training area, functional fitness space, dedicated yoga studio, spin class room, swimming pool (25m), sauna and steam rooms, juice bar, and locker rooms with shower facilities. Our equipment is replaced or upgraded every 3 years to ensure members have access to the latest fitness technology.",
        metadata={"source": "facilities.txt"}
    )
]

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
db = Chroma.from_documents(docs, embedding_function)
retriever = db.as_retriever(search_type="mmr", search_kwards = {"k":3})

In [11]:
retriever_tool = create_retriever_tool(
    retriever, "retriever_tool", "Information related to Gym History & Founder, Operating Hours, Membership Plans, Fitness Classes, Personal Trainers, and Facilities & Equipment of Peak Performance Gym",
)

@tool
def off_topic():
    """Catch all Questions NOT related to Peak Performance Gym's history, hours, membership plans, fitness classes, trainers, or facilities"""
    return "Forbidden - do not respond to the user"

tools = [retriever_tool, off_topic]

In [12]:
from typing import Annotated, Literal, TypedDict, Sequence
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import StateGraph, add_messages, END, START
from langgraph.prebuilt import ToolNode

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]


In [13]:
def agent(state):
    messages = state["messages"]
    model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
    model = model.bind_tools(tools)
    response = model.invoke(messages)
    return {"messages": [response]}

def should_continue(state) -> Literal["tools", END]:
    last_msg = state["messages"][-1]
    if last_msg.tool_calls:
        return "tools"
    else:
        return END


In [14]:
workflow = StateGraph(AgentState)

AGENT = "agent"
TOOLS= "tools"

workflow.add_node(AGENT, agent)
tool_node = ToolNode(tools)
workflow.add_node(TOOLS, tool_node)

workflow.add_edge(START, AGENT)
workflow.add_conditional_edges(AGENT, should_continue)
workflow.add_edge(TOOLS, AGENT)

graph = workflow.compile()

In [16]:
print(graph.get_graph().draw_mermaid())

%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	agent(agent)
	tools(tools)
	__end__([<p>__end__</p>]):::last
	__start__ --> agent;
	tools --> agent;
	agent -.-> tools;
	agent -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [ ]:
# from IPython.display import Image, display
# from langchain_core.runnables.graph import MermaidDrawMethod
#
# display(
#     Image(
#         graph.get_graph().draw_mermaid_png(
#             draw_method=MermaidDrawMethod.API,
#         )
#     )
# )

In [18]:
graph.invoke(
    input={"messages": [HumanMessage(content="How will the weather be tommorrow?")]}
)

{'messages': [HumanMessage(content='How will the weather be tommorrow?', additional_kwargs={}, response_metadata={}, id='517cc580-5d3a-4b84-9ca5-c3e481c86253'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'off_topic', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-8b5a11a6-cabf-45d4-982a-0f73a8ac23a3-0', tool_calls=[{'name': 'off_topic', 'args': {}, 'id': '39c7f7ec-de5f-47fd-b7a2-9c3526d88405', 'type': 'tool_call'}], usage_metadata={'input_tokens': 121, 'output_tokens': 10, 'total_tokens': 190, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='Forbidden - do not respond to the user', name='off_topic', id='a4abae96-6d32-4b9a-ba3c-0a1f9f37436b', tool_call_id='39c7f7ec-de5f-47fd-b7a2-9c3526d88405'),
  AIMessage(content='I am sorry, but I can only answer questions related to Peak Performance Gym. I cannot provide weather forecasts.', addit

In [21]:

graph.invoke(input={
    "messages": [HumanMessage(content="Who is the owner and what are the timings?")]
})

{'messages': [HumanMessage(content='Who is the owner and what are the timings?', additional_kwargs={}, response_metadata={}, id='acff5cb2-d61e-4529-89ec-adb1300e77b8'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'retriever_tool', 'arguments': '{"query": "gym timings"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-8316e3bf-bd42-4ee0-8416-d5cb83299550-0', tool_calls=[{'name': 'retriever_tool', 'args': {'query': 'owner of the gym'}, 'id': 'f982ae3c-5c75-48c5-8199-d6a0cb9115ed', 'type': 'tool_call'}, {'name': 'retriever_tool', 'args': {'query': 'gym timings'}, 'id': '887a93a9-527e-47e6-b406-26655eee61c0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 122, 'output_tokens': 36, 'total_tokens': 253, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="Peak Performance Gym was founded in 2015 by former Olympic athlete Marcus Chen. With over 15 years of